In [1]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import train
import preprocess

In [2]:
questions = {
}

questions[1]=preprocess.string_to_analogy("land:excavate::shoal:_,sound salvage dredge survey sidle,dredge")
#questions[2]=preprocess.string_to_analogy("sword:brandish::_:_,")
questions[3]=preprocess.string_to_analogy("volcano:quiescent::talent:_,imperious hyperbolical oblique latent pliant,latent")
questions[4]=preprocess.string_to_analogy("style:flamboyant::behavior:_,brazen lofty volatile insolent sassy,brazen")
questions[5]=preprocess.string_to_analogy("direct:confront::oblique:_,unsettle incite sidle stymie flourish,sidle")
questions[6]=preprocess.string_to_analogy("beforehand:trepidation::afterwards:_,bravado decadence hyperbole foolhardy rue,rue")
questions[7]=preprocess.string_to_analogy("person:odious::action:_,unsettling imperious heinous lofty haughty,heinous")
questions[8]=preprocess.string_to_analogy("naughtiness:permit::misbehavior:_,dredge confront pique countenance stymie,countenance")
questions[9]=preprocess.string_to_analogy("speech:hyperbole::behavior:_,trepidation quiescence bravado pique parsimony,bravado")
questions[10]=preprocess.string_to_analogy("emotional:resistance::physical:_,cadaver survey conflagration decadence friction,friction")
questions[11]=preprocess.string_to_analogy("interest:rouse::curiosity:_,assuage nettle dredge rue pique,pique")
questions[12]=preprocess.string_to_analogy("mien:haughty::countenance:_, flamboyant imperious befuddled canny brazen,imperious")
questions[13]=preprocess.string_to_analogy("threateningly:brandish::flamboyantly:_,confront flaunt fan incite garner,flaunt")
#questions[14]=preprocess.string_to_analogy
questions[15]=preprocess.string_to_analogy("latent:possibility::hidden:_,cache ire hovel visage urchin,cache")
#questions[16]=preprocess.string_to_analogy
questions[17]=preprocess.string_to_analogy("travel:blocked::attempt:_,rued incited salvage stymied unsettled,stymied")
questions[18]=preprocess.string_to_analogy("salesman:canny::businesswoman:_,entrepreneurial prolific shrewd haughty frugal,shrewd")
questions[19]=preprocess.string_to_analogy("brave:reckless::frugal:_,prolific audacious foolhardy poor parsimonious,parsimonious")
questions[20]=preprocess.string_to_analogy("personality:pliant::behavior:_,lofty insolent cooperative volatile popular,cooperative")



['land', 'excavate', 'shoal', ['sound', 'salvage', 'dredge', 'survey', 'sidle'], 'dredge']
['volcano', 'quiescent', 'talent', ['imperious', 'hyperbolical', 'oblique', 'latent', 'pliant'], 'latent']
['style', 'flamboyant', 'behavior', ['brazen', 'lofty', 'volatile', 'insolent', 'sassy'], 'brazen']
['direct', 'confront', 'oblique', ['unsettle', 'incite', 'sidle', 'stymie', 'flourish'], 'sidle']
['beforehand', 'trepidation', 'afterwards', ['bravado', 'decadence', 'hyperbole', 'foolhardy', 'rue'], 'rue']
['person', 'odious', 'action', ['unsettling', 'imperious', 'heinous', 'lofty', 'haughty'], 'heinous']
['naughtiness', 'permit', 'misbehavior', ['dredge', 'confront', 'pique', 'countenance', 'stymie'], 'countenance']
['speech', 'hyperbole', 'behavior', ['trepidation', 'quiescence', 'bravado', 'pique', 'parsimony'], 'bravado']
['emotional', 'resistance', 'physical', ['cadaver', 'survey', 'conflagration', 'decadence', 'friction'], 'friction']
['interest', 'rouse', 'curiosity', ['assuage', 'ne

In [3]:
correct = []
incorrect = []


In [4]:
file = './GoogleNews-vectors-negative300.bin'

print(f"Training from {file}")
model = KeyedVectors.load_word2vec_format(file, binary=True)
model.init_sims(replace=True)
print("Model trained")

Training from ./GoogleNews-vectors-negative300.bin
Model trained


In [5]:
for question_number in questions:
    data = questions[question_number] 
    print(f"{question_number}. {data[0]}:{data[1]}::{data[2]}:_____?")
    for answer in data[3]:
        print(f"\t{answer}")

    ref = model.most_similar_cosmul(positive=[data[1],data[2]], negative=[data[0]], topn=5)
    print(f"Predictions are {ref}")

    for word in data[3]:
        if word not in model.vocab:
            data[3].remove(word)

    submission = model.most_similar_to_given(ref[0][0], data[3])
    print()
    print(f"Answer: {submission}")
    print(f"{data[0]}:{data[1]}::{data[2]}:{submission}")
    if submission ==  data[4]:
        print("Answer is correct.")
        correct.append(question_number)
    else:
        print(f"Answer is incorrect, intended answer was {data[4]}.")
        incorrect.append(question_number)

    print("\n\n")

1. land:excavate::shoal:_____?
	sound
	salvage
	dredge
	survey
	sidle
Predictions are [('snailfish', 0.8898302912712097), ('dredge', 0.8836666941642761), ('oarfish', 0.8802536725997925), ('divers', 0.8777430653572083), ('finback_whale', 0.8762484192848206)]

Answer: dredge
land:excavate::shoal:dredge
Answer is correct.




3. volcano:quiescent::talent:_____?
	imperious
	hyperbolical
	oblique
	latent
	pliant
Predictions are [('talented', 0.8820160031318665), ('supremely_talented', 0.845802903175354), ('immensely_talented', 0.8376045227050781), ('bare_cupboard', 0.818310022354126), ('brainpower', 0.8174195885658264)]

Answer: pliant
volcano:quiescent::talent:pliant
Answer is incorrect, intended answer was latent.




4. style:flamboyant::behavior:_____?
	brazen
	lofty
	volatile
	insolent
	sassy
Predictions are [('antisocial', 0.9095085263252258), ('behaviors', 0.8813495635986328), ('sexually_predatory', 0.878678023815155), ('Tealeaf_CEM_solutions', 0.8783724904060364), ('behaving', 0.876

In [6]:
print(f"Model scored {len(correct)} out of {len(correct)+len(incorrect)}!")    

Model scored 8 out of 17!


In [32]:
sum = model[questions[3][1]] + model[questions[3][2]] - model[questions[3][0]]
model.similar_by_vector(sum, topn=5)

[('talent', 0.472327321767807),
 ('quiescent', 0.44346877932548523),
 ('talented', 0.4036876857280731),
 ('supremely_talented', 0.3586314916610718),
 ('immensely_talented', 0.3515331745147705)]

In [29]:
model.n_similarity(["shoal"],["dredge"])

0.43346363

In [34]:
from sklearn.neighbors import KDTree
class ANNSearch:
    word2idx = {}
    idx2word = {}
    data = []

    def __init__(self, model):
        for counter, key in enumerate(model.vocab.keys()):
            self.data.append(model[key])
            self.word2idx[key] = counter
            self.idx2word[counter] = key

        # leaf_size is a hyperparameter
        self.data = np.array(self.data)
        self.tree = KDTree(self.data, leaf_size=100)
        
    def search_by_vector(self, v, k=10):
        dists, inds = self.tree.query([v], k)
        return zip(dists[0], [self.idx2word[idx] for idx in inds[0]])

    def search(self, query, k=10):
        vector = self.data[self.word2idx[query]]
        return self.search_by_vector(vector, k)

ModuleNotFoundError: No module named 'sklearn'